<a href="https://colab.research.google.com/github/salman7636/Clinical-Notes-Classification/blob/main/Clinical.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Deep Learning Clinical Notes Classification
# Name: Salman Firdous

import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib


# 1. Load dataset


data = pd.read_csv("clinical_notes_dl.csv")


# 2. Text cleaning


def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

data["clean_text"] = data["text"].apply(clean_text)



# 3. Label Encoding


label_encoder = LabelEncoder()
data["encoded_label"] = label_encoder.fit_transform(data["label"])

joblib.dump(label_encoder, "label_encoder.pkl")

print("\nClass Distribution:")
print(data["encoded_label"].value_counts())


# 4. Train-Test Split


X = data["clean_text"]
y = data["encoded_label"]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

print("\nTrain size:", len(X_train))
print("Test size:", len(X_test))


# 5. Tokenization


tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

joblib.dump(tokenizer, "tokenizer.pkl")

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

max_len = 80

X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')


# 6. LSTM Model


model = Sequential([
    Embedding(input_dim=5000, output_dim=128, input_length=max_len),
    Bidirectional(LSTM(64)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(len(np.unique(y_train)), activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()



# 7. Train model


history = model.fit(
    X_train_pad,
    y_train,
    epochs=25,
    batch_size=4,
    validation_split=0.2,
    verbose=1
)


# 8. Evaluation


y_pred = np.argmax(model.predict(X_test_pad), axis=1)

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred, zero_division=0))


# 9. Save model (NEW FORMAT)


model.save("clinical_notes_lstm_model.keras")
print("\nModel saved successfully as clinical_notes_lstm_model.keras")


# 10. Prediction Function


def predict_note(text):
    cleaned = clean_text(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    pad = pad_sequences(seq, maxlen=max_len, padding='post')
    pred = model.predict(pad)
    label = label_encoder.inverse_transform([np.argmax(pred)])
    return label[0]

# Example test
example = "patient advised insulin for diabetes management"
print("\nExample Prediction:", predict_note(example))



Class Distribution:
encoded_label
0    4
1    4
3    4
4    4
2    4
Name: count, dtype: int64

Train size: 14
Test size: 6


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 227ms/step - accuracy: 0.2159 - loss: 1.6159 - val_accuracy: 0.3333 - val_loss: 1.6024
Epoch 2/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.2614 - loss: 1.6108 - val_accuracy: 0.6667 - val_loss: 1.6114
Epoch 3/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.2614 - loss: 1.6047 - val_accuracy: 0.0000e+00 - val_loss: 1.6130
Epoch 4/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.4773 - loss: 1.5893 - val_accuracy: 0.0000e+00 - val_loss: 1.6225
Epoch 5/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.4460 - loss: 1.5739 - val_accuracy: 0.0000e+00 - val_loss: 1.6361
Epoch 6/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.2926 - loss: 1.5809 - val_accuracy: 0.0000e+00 - val_loss: 1.6503
Epoch 7/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.4148 - loss: 1.5672 - val_accuracy: 0.0000e+00 - val_loss: 1.6609
Epoch 8/25
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.3068 - loss: 1.5462 - val_accuracy: 0.000